<a href="https://colab.research.google.com/gist/gowatana/1fc66e48db015732e3a88870af2ac67b/demo-tensorflow-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考：初心者のための TensorFlow 2.0 入門
* https://www.tensorflow.org/tutorials/quickstart/beginner?hl=ja


# 1. 画像判別モデルの作成（手動実行の例）

TensorFlowのPythonライブラリをインポート

In [ ]:
import tensorflow as tf

TensorFlowのバージョン確認

In [ ]:
print(tf.__version__)

## 1-1. サンプル データセット（MNIST）をダウンロード

In [3]:
mnist = tf.keras.datasets.mnist

## 1-2. 学習にむけて、データの前処理を実施

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 1-3. モデルの学習と保存

モデルのニューラル ネットワークを定義

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

モデルを作成（コンパイル）

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

モデルの学習（5回繰り返す）と保存

In [ ]:
model.fit(x_train, y_train, epochs=5)
model.save('./mnist_saved_model')

保存されたモデルの確認（Linuxのlsコマンド）

In [ ]:
!ls -l ./mnist_saved_model

## 1-4. モデルのテスト

テスト用のデータセットを利用して、作成されたモデルのテストを実施

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

# ここまでのPythonコードのまとめ
ここまでの手順をまとめて、ひとつのコードとして実行する。

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test, verbose=2)

# 2. モデルによる推論の実行（MNIST画像の判別）

※ただし、トレーニングで利用したMNIST画像を流用しているため、ほぼ確実に正解します。

## 2-1. 推論用データの用意

MNISTデータセットから10件を、画像ファイル（PNG）として保存

In [10]:
from PIL import Image
import tensorflow as tf
import os

dir_path = 'data/images'

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 保存先のディレクトリを作成
os.makedirs(dir_path, exist_ok=True)

# 画像の保存
for i in range(10):
    img = Image.fromarray(x_train[i])
    img.save(os.path.join(dir_path, f'test_{i}.png'))

10件の画像ファイルが保存されたことを確認

In [ ]:
!ls data/images/

画像の表示確認

コードの1行目でファイルを指定する。

* 例1： test_0.png → 「5」
* 例2： test_1.png → 「0」
* 例3： test_2.png → 「4」

In [ ]:
img_file_name = 'data/images/test_0.png'

import matplotlib.pyplot as plt
from PIL import Image
img = Image.open(img_file_name)
plt.imshow(img, cmap='gray')  # 画像がグレースケールの場合は cmap='gray' を使用
plt.show()

## 2-2. モデルによる推論の実行

生成したモデルで、直前に表示した画像に書かれている数字を判別します。

※直前のコード実行で、img_file_name = 'data/images/test_0.png' を指定してあります。

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf

# 画像の読み込み
img = Image.open(img_file_name)

# 前処理として、モデルの入力形式に合わせるために
# 画像のグレースケール変換、サイズ変更などを行う
img = img.convert('L').resize((28, 28))
img_array = np.array(img) / 255.0
img_array = img_array.reshape(1, 28, 28)

# 保存されているモデルのロード
model = tf.keras.models.load_model('./mnist_saved_model')

# 推論の実行
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print("Number:", predicted_class)